In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from utils.metrics import Metrics_df,print_metrics, precision, recall, f1
from Config import config
import mlflow
import mlflow.lightgbm
from sklearn.model_selection import train_test_split
import glob
from tqdm.auto import tqdm
from utils.trace import reduce_mem_usage
import numpy as np
from cuml.ensemble import RandomForestClassifier as cuRFC


In [2]:


import pandas as pd
from sklearn.metrics import roc_auc_score
from Config import config
import warnings
warnings.filterwarnings("ignore")
import optuna  # pip install optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc
import pickle
import tempfile

# Loading data and group split

In [3]:
# df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data.csv")
# df_pairs = df_pairs.sample(frac=1).reset_index(drop=True)
# kf = GroupKFold(n_splits=5)
# for i, (trn_idx, val_idx) in enumerate(kf.split(df_pairs, df_pairs["placeId1"], df_pairs["placeId1"])):
#     df_pairs.loc[val_idx, "fold"] = i
# df_pairs["fold"].value_counts()
# df_pairs.to_csv("/workspace/Training_data/training/Full_training_data_fold.csv",index=None)

# Loading data

In [4]:
df_pairs = pd.read_csv("/workspace/Traininf_model_v4/Training_data/Full_training_data_fold.csv")

In [5]:
features = ['similarity', 'haversine', 'name_davies',
       'name_leven', 'name_dice', 'name_jaro', 'name_set_ratio',
       'street_davies', 'street_leven', 'street_jaro', 'email_davies',
       'email_leven', 'email_jaro', 'url_davies', 'url_leven', 'url_jaro',
       'brands_davies', 'brand_leven', 'brand_jaro', 'phone_lcs',
       'subcat_WRatio', 'subcat_ratio', 'subcat_token_set_ratio',
       'Is_direction_match_0', 'Is_direction_match_1', 'Is_direction_match_2',
       'Is_house_match_0', 'Is_house_match_1', 'Is_house_match_2',
       'Is_category_match_0', 'Is_category_match_1', 'Is_subcategory_match_0',
       'Is_subcategory_match_1', 'Is_subcategory_match_2', 'Is_brand_match_0',
       'Is_brand_match_1', 'Is_brand_match_2', 'Is_brand_match_3',
       'Is_related_cat_0', 'Is_related_cat_1', 'Is_name_number_match_0',
       'Is_name_number_match_1', 'Is_name_number_match_2',
       'Is_name_number_match_3', 'is_phone_match_1', 'is_phone_match_2',
       'is_phone_match_3', 'is_phone_match_4', 'Is_email_match_0',
       'Is_email_match_1', 'Is_email_match_2', 'Is_url_match_0',
       'Is_url_match_1', 'Is_url_match_2']

In [6]:

def objective(trial, df_pairs):
    param_grid = {
        "split_criterion": trial.suggest_int("split_criterion", 0,1),
        "max_depth": trial.suggest_int("max_depth", 4, 24),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1,60,step=1),
        "min_samples_split": trial.suggest_int("min_samples_split", 2,30,step=1),
        "n_bins": trial.suggest_int("n_bins", 8,256,step=8),
        'n_estimators': trial.suggest_int("n_estimators", 100,1200,step=100),
    }
    cv_scores = np.empty(5)
    cv_scores_train = np.empty(5)
    for fold in [0, 1, 2, 3, 4]:
        X_train = df_pairs[df_pairs["fold"] != fold]
        X_test = df_pairs[df_pairs["fold"] == fold]
    

        model = cuRFC(verbose=100,**param_grid,max_batch_size=8164,n_streams=10)   
    
        model.fit(X_train[features],X_train["duplicate_flag"])
        preds = model.predict_proba(X_test[features])[:, 1]
        preds_train = model.predict_proba(X_train[features])[:, 1]
        cv_scores[fold] = roc_auc_score(X_test["duplicate_flag"], preds)
        cv_scores_train[fold] = roc_auc_score(X_train["duplicate_flag"],preds_train)
    print(f"train auc score = {np.mean(cv_scores_train)}")
    print(f"test auc score = {np.mean(cv_scores)}")
    return np.mean(cv_scores)

In [7]:
date_log = datetime.now().strftime("%Y_%m_%d_%H_%M")
study = optuna.create_study(direction="maximize", study_name=f"parameter_tuning_{config.country}_{date_log}")
func = lambda trial: objective(trial, df_pairs)
study.optimize(func, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2022-12-11 15:25:26,001] A new study created in memory with name: parameter_tuning_GBR_2022_12_11_15_25


[D] [15:26:02.042631] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [15:26:34.501950] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [15:27:06.789211] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [15:27:39.040933] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.


[I 2022-12-11 15:28:10,336] Trial 0 finished with value: 0.9996428843749028 and parameters: {'split_criterion': 0, 'max_depth': 24, 'min_samples_leaf': 59, 'min_samples_split': 14, 'n_bins': 160, 'n_estimators': 1000}. Best is trial 0 with value: 0.9996428843749028.


train auc score = 0.9997532015848589
test auc score = 0.9996428843749028
[D] [15:28:11.288168] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [15:28:18.709937] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [15:28:26.063084] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [15:28:33.614923] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Convert

[I 2022-12-11 15:28:47,336] Trial 1 finished with value: 0.9991736872523909 and parameters: {'split_criterion': 0, 'max_depth': 9, 'min_samples_leaf': 38, 'min_samples_split': 17, 'n_bins': 144, 'n_estimators': 200}. Best is trial 0 with value: 0.9996428843749028.


...

train auc score = 0.9999806463920713
test auc score = 0.999809404747675
[D] [18:21:42.185950] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [18:22:06.864919] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [18:22:31.474640] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [18:22:55.994824] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converti

[I 2022-12-11 18:23:43,993] Trial 98 finished with value: 0.9996972205230049 and parameters: {'split_criterion': 1, 'max_depth': 24, 'min_samples_leaf': 50, 'min_samples_split': 7, 'n_bins': 168, 'n_estimators': 900}. Best is trial 97 with value: 0.999809404747675.


train auc score = 0.9998022746804616
test auc score = 0.9996972205230049
[D] [18:23:44.899024] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [18:24:12.822027] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [18:24:40.799285] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
[D] [18:25:08.821715] /workspace/.conda-bld/work/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, but got the opposite. Convert

[I 2022-12-11 18:26:03,460] Trial 99 finished with value: 0.9998090635063652 and parameters: {'split_criterion': 1, 'max_depth': 24, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_bins': 160, 'n_estimators': 1000}. Best is trial 97 with value: 0.999809404747675.


train auc score = 0.9999775077285349
test auc score = 0.9998090635063652
Number of finished trials: 100
Best trial: {'split_criterion': 1, 'max_depth': 24, 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_bins': 160, 'n_estimators': 900}


In [8]:
study.trials_dataframe().to_csv("/workspace/randomforest_trials1.csv",index=None)